In [ ]:
from urllib.parse import unquote
import re


from IPython.display import Image
from playwright.async_api import async_playwright

In [ ]:
# AIRPORT = ("Vancouver", "YVR")
AIRPORT = ("Orlando", "MCO")
DATES = ("15 Aug", "21 Aug")

In [ ]:
def extract_numeric_and_non_numeric(s):
    # Find all non-numeric characters (prefix and suffix)
    non_numeric = ''.join(re.findall(r'\D+', s))
    # Find all numeric characters
    numeric = ''.join(re.findall(r'\d+', s))    
    return numeric, non_numeric

In [ ]:
def insert_space_before_capital(s):
    return re.sub(r'(?<!\s)(?=[A-Z])', ' ', s)

In [ ]:
async def display_screenshot():
    return Image(await page.screenshot(path="test-page.png"))

In [ ]:
playwright = await async_playwright().start()

In [ ]:
browser = await playwright.chromium.launch()
page = await browser.new_page()

In [ ]:
await page.goto("https://www.google.com/travel/flights/search")
await display_screenshot()

In [ ]:
where_to = page.get_by_placeholder("Where to?")
await where_to.press_sequentially(AIRPORT[0])
await page.wait_for_timeout(2000)
await display_screenshot()

In [ ]:
airport = page.get_by_text(AIRPORT[1])
await airport.click()
await display_screenshot()

In [ ]:
departure = page.get_by_placeholder("Departure").nth(0)
await page.wait_for_timeout(500)
await departure.fill(DATES[0])
await page.wait_for_timeout(100)
await departure.press("Enter")
await page.wait_for_timeout(100)
await display_screenshot()

In [ ]:
departure = page.get_by_placeholder("Return").nth(0)
await page.wait_for_timeout(100)
await departure.fill(DATES[1])
await page.wait_for_timeout(100)
await departure.press("Enter")
await page.wait_for_timeout(100)
await display_screenshot()

In [ ]:
await page.wait_for_timeout(2000)
await page.keyboard.press("Escape")
is_explore_button = await page.get_by_label("Explore", exact=True).bounding_box()
if is_explore_button is not None:
    search = page.get_by_label("Explore", exact=True)
else:
    search = page.get_by_label("Search", exact=True)
await page.wait_for_timeout(100)
await search.click()
await page.wait_for_timeout(2000)
await display_screenshot()

In [ ]:
filters = page.get_by_label("All filters")
await filters.click()
await page.wait_for_timeout(1000)

hide_separate = page.get_by_text("Hide separate tickets")
await hide_separate.scroll_into_view_if_needed()
await hide_separate.click()
await page.wait_for_timeout(1000)
await page.keyboard.press("Escape")

await display_screenshot()

In [ ]:
data = []
for row in await page.get_by_role("listitem").all():
    texts = (await row.all_inner_texts())[0].split('\n')
    if "more flights" in texts[0]:
        continue
    nonstop = False
    if "nonstop" in texts[6].lower():
        nonstop = True

    price_str = texts[-2] if texts[-2] != "round trip" else texts[-3]
    price_components = extract_numeric_and_non_numeric(price_str)

    parsed_texts = {
        "departure": " ".join(texts[0].split()),
        "landing": " ".join(texts[2].split()),
        "airline": insert_space_before_capital(texts[3]),
        "duration": texts[4],
        "nonstop?": nonstop,
        "price_currency": price_components[1],
        "price_unit": price_components[0],
    }
    if not nonstop:
        parsed_texts['stops'] = texts[7]
    data.append(parsed_texts)
    print((await row.all_inner_texts())[0])
    print("-----------")

print(data)

In [ ]:
data

In [ ]:
print(data)

In [ ]:
await page.keyboard.press("Escape")

In [ ]:
await display_screenshot()